In [ ]:
import os
import cv2
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
#from tqdm import tqdm

In [ ]:
DATA_PATH = r'/scratch/gallowaa/cciw/Data'
SAVE_PATH = '/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Test/Lab'

In [ ]:
# Search for all image files in testing set...
all_images = glob(os.path.join(DATA_PATH,'Videos_and_stills/TestingSet/Lab/*/*/*/Images/Quad*/*.jpg'))
len(all_images)

In [ ]:
im1   = cv2.imread(all_images[0])
rgb1  = cv2.resize(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB), (680, 420))
img1 = cv2.cvtColor(rgb1, cv2.COLOR_RGB2GRAY)

In [ ]:
im2   = cv2.imread(all_images[22])
rgb2  = cv2.resize(cv2.cvtColor(im2, cv2.COLOR_BGR2RGB), (680, 420))
img2 = cv2.cvtColor(rgb2, cv2.COLOR_RGB2GRAY)

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()

In [ ]:
kp1, des1 = sift.detectAndCompute(rgb1, None)
kp2, des2 = sift.detectAndCompute(rgb2, None)

In [ ]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

In [ ]:
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1, des2, k=2)

In [ ]:
good = []
pts1 = []
pts2 = []

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)

In [ ]:
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)

# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

In [ ]:
#rgb1.shape

In [ ]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0, y0 = map(int, [0, -r[2]/r[1] ])
        x1, y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color, 1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

In [ ]:
# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2,F)
lines1 = lines1.reshape(-1, 3)
img5, img6 = drawlines(img1, img2, lines1, pts1, pts2)

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3, img4 = drawlines(img2, img1, lines2, pts2, pts1)

plt.figure(figsize=(16, 8))
plt.subplot(121), plt.imshow(img5)
plt.subplot(122), plt.imshow(img3)
plt.show()